In [1]:
import pandas as pd
import numpy as np
import random
from os import listdir
from os.path import isfile, join,getsize,isdir
import csv
import numpy
import cPickle as pickle
import json
from random import shuffle
import warnings
import time
import msgpack
import re
import matplotlib.pyplot as plt
from datetime import date
import math
import pylab

path = "C:\Users\Data Science 5\Desktop\L_equipe"

#n'affiche pas les warnings
warnings.filterwarnings('ignore')
#affiche la totalité des colonnes du dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

### Utils

In [2]:
def make_autopct(values):
    def my_autopct(pct):
        total = sum(values)
        val = int(round(pct*total/100.0))
        if val > 10000 :
            val = str(val/1000) + "K"
            return '{p:.2f}% ({v})'.format(p=pct,v=val)
        return '{p:.2f}% ({v:d})'.format(p=pct,v=val)
    return my_autopct

In [3]:
df_prod42_test2 =  pd.read_pickle(path+"\s_temp_dataframe_sub_2015.pkl")
#----------------------------graphe des nouveaux abonnés 2015   OK ---------------------------------------------

df_equipe_payant = df_prod42_test2.loc[df_prod42_test2.service == 'Abo_Equipe_payant' ]
df_equipe_gratuit = df_prod42_test2.loc[df_prod42_test2.service == 'Abo_Equipe_gratuit' ] 
df_viatiers_payant = df_prod42_test2.loc[df_prod42_test2.service == 'Abo_via_tiers_payant' ]
df_viatiers_gratuit = df_prod42_test2.loc[df_prod42_test2.service == 'Abo_via_tiers_gratuit' ]

fig = plt.figure(figsize=(13,5.8))
liste = ['janvier','fevrier','mars', 'avril', 'mai', 'juin']

ax = fig.add_subplot(111) 

width = 0.2
cmap=plt.cm.prism
cs = cmap(np.linspace(0., 1.5, 10))

df_equipe_payant.groupby(df_equipe_payant.sub_month).ClientUserId.nunique().plot(kind='bar'
                                        ,color=cs[0], ax=ax,  width=width, position=1)
df_equipe_gratuit.groupby(df_equipe_gratuit["sub_month"]).ClientUserId.nunique().plot(kind='bar'
                                        , color=cs[1] ,ax=ax,  width=width, position=0)
df_viatiers_payant.groupby(df_viatiers_payant["sub_month"]).ClientUserId.nunique().plot(kind='bar'
                                        ,color=cs[2], ax=ax,  width=width, position=2)
df_viatiers_gratuit.groupby(df_viatiers_gratuit["sub_month"]).ClientUserId.nunique().plot(kind='bar'
                                        , color=cs[3],ax=ax,  width=width, position=3)
ax.set_xticklabels( liste)
pylab.axes().set_xlabel('')
ax.set_xlim(-0.7,5.3)
ax.grid(zorder=0)
#
plt.tight_layout()
ax.legend(['abonnement Equipe payant', 'abonnement Equipe gratuit', 'via tiers payant', 'via tiers gratuit'])
ax.set_title("nouveaux abonnements 2015")
plt.show()
#plt.savefig(path+"\sub_equipe_payant_gratuit.png")

In [4]:
df_prod42_test2 = pd.read_pickle(path+"\s_final_groupby_dataframe_sub_2015.pkl")
#----------------------------répartition des nouveaux abonnés 2015 OK --------------------------------

df_test =df_prod42_test2.ix[:,['ClientUserId','service_Abo_Equipe_gratuit','service_Abo_Equipe_payant','service_Abo_via_tiers_gratuit'
                          ,'service_Abo_via_tiers_payant']]

df_test = df_test.groupby('ClientUserId')['service_Abo_Equipe_gratuit','service_Abo_Equipe_payant','service_Abo_via_tiers_gratuit'
                          ,'service_Abo_via_tiers_payant'].sum()

df_test = df_test.apply(pd.Series.value_counts)
df_test = df_test.loc[1:].sum()

plt.rcParams['font.size'] = 12.0
taille = df_test.shape[0]

explode= (0,0.12,0,0)

cmap=plt.cm.prism

cs=plt.cm.Set1(0.6*np.arange(taille)/taille)

fig = plt.figure(figsize=(6,6))
ax = plt.subplot(1,1,1)

df_test.plot(kind='pie' 
        , colors=cs ,autopct=make_autopct(df_test)
        , labels=['abo Equipe gratuit', 'abo Equipe payant', 'abo via tiers gratuit', 'abo via tier payant']
        ,labeldistance=1.03, pctdistance=0.65, explode=explode, startangle=160)
pylab.axes().set_ylabel('')
plt.axis('equal')
ax.set_title("repartition nouveaux abonnes 2015")
plt.show()

In [11]:
df_prod42_test2 = pd.read_pickle(path+"\s_final_groupby_dataframe_sub_2015.pkl")
#-------------------------- pie pour le provider des abonnés de l'equipe  OK --------------------------
df_prod42_test2["provider2"] = df_prod42_test2.provider
df_prod42_test2["newprovider"] = df_prod42_test2.groupby(["provider"]).ClientUserId.transform(lambda x : 1.0*x.count()/len(df_prod42_test2)*100)
df_prod42_test2.loc[df_prod42_test2['newprovider'] < 0.8, 'provider2'] = 'other'
df_prod42_test2 = df_prod42_test2[['ClientUserId','provider2']]
df_prod42_test2 = df_prod42_test2.drop_duplicates()

taille = len(df_prod42_test2.provider2.value_counts())
cs=plt.cm.Set1(1.0*np.arange(taille)/taille)

fig = plt.figure()
ax = plt.subplot(111)
values = df_prod42_test2.provider2.value_counts()

df_prod42_test2.provider2.value_counts().plot(kind='pie' 
        ,colors=cs ,autopct=make_autopct(values),labeldistance=1.05, radius=1.1, pctdistance=0.75, startangle=20)
pylab.axes().set_ylabel('')
plt.axis('equal')
ax.set_title('provider des nouveaux subs de 2015')
plt.show()
#plt.savefig(path+"\sub_provider")

In [12]:
df_prod42_test2 = pd.read_pickle(path+"\s_final_groupby_dataframe_sub_2015.pkl")
#-------------------------- pie pour le statut des abonnés de l'equipe  OK --------------------------

df_test =df_prod42_test2.ix[:,["SubStatus_Active_Subscription","SubStatus_Cancelled_By_AutoRenew_Process"
        ,"SubStatus_Cancelled_By_Customer_Support_Agent"
        ,"SubStatus_Cancelled_By_User","SubStatus_Expired_Subscription","SubStatus_Failure_Retry_Mode"]]
df_test = df_test.apply(pd.Series.value_counts)
df_test = df_test.loc[1]

#df_prod42_test2["substat"] = df_prod42_test2.SubscriptionStatus
#df_prod42_test2["sub"] = df_prod42_test2.groupby(["SubscriptionStatus"]).ClientUserId.transform(lambda x : 1.0*x.count()/len(df_prod42_test2)*100)
#df_prod42_test2.loc[df_prod42_test2['sub'] < 0.9, 'substat'] = 'other'
#plt.rcParams['font.size'] = 10.0

taille = df_test.shape[0]
cs=plt.cm.Set1(1.0*np.arange(taille)/taille)

fig = plt.figure(figsize=(6,6))
ax = plt.subplot(111)
values = df_test

df_test.plot(kind='pie' 
        ,colors = cs, autopct=make_autopct(values)
        ,labels=["Active Subscription","Cancelled By AutoRenew Process","Cancelled By Customer Support Agent"
        ,"Cancelled By User","Expired Subscription","Failure Retry Mode"]
        ,labeldistance=1.1, pctdistance=0.75, radius=0.2)
pylab.axes().set_ylabel('')
ax.set_title('status des subs')

plt.axis('equal')
plt.tight_layout()
plt.show()
#plt.savefig(path+"\sub_account_satus")

In [14]:
df_prod42_test2 = pd.read_pickle(path+"\s_final_groupby_dataframe_sub_2015.pkl")
#-------------------------- pie pour le prix mensuel de l'abonnement OK--------------------------
df_prod42_test2 = df_prod42_test2.loc[((df_prod42_test2.service_Abo_Equipe_payant ==1))] 
                                       #|(df_prod42_test2.service_Abo_via_tiers_payant ==1))]

df_prod42_test2["ExplicitPrice2"] = df_prod42_test2.prix_mensuel
df_prod42_test2["newprice"] = df_prod42_test2.groupby(["prix_mensuel"]).ClientUserId.transform(lambda x : 1.0*x.count()/len(df_prod42_test2)*100)
df_prod42_test2.loc[df_prod42_test2['newprice'] < 0.9, 'ExplicitPrice2'] = 'other'

plt.rcParams['font.size'] = 10.0
taille = len(df_prod42_test2.ExplicitPrice2.value_counts())
cs=plt.cm.Set1(0.1+0.65*np.arange(taille)/taille)

values = (df_prod42_test2.ExplicitPrice2.value_counts())

fig = plt.figure(figsize=(6,6))
ax = plt.subplot(111)

df_prod42_test2.ExplicitPrice2.value_counts(sort = False).plot(kind='pie' 
       ,colors = cs, autopct=make_autopct(values),labeldistance=1.02, pctdistance=0.75,radius=1.02)
pylab.axes().set_ylabel('')
ax.set_title('prix du service de souscription pour les abonnes payant(en euros)')

plt.axis('equal')
plt.tight_layout()
plt.show()
##plt.savefig(path+"\sub_account_satus")

In [15]:
df_prod42_test2 = pd.read_pickle(path+"\s_final_groupby_dataframe_sub_2015.pkl")
#-------------------------- pie pour la durée l'abonnement OK--------------------------

df_prod42_test2["duree2"] = df_prod42_test2.duree_abonnement_reelle
df_prod42_test2["duree_souscription"] = df_prod42_test2.duree_abonnement_reelle

df_prod42_test2.duree2 = np.nan
df_prod42_test2.loc[df_prod42_test2['duree_souscription'] < 9, 'duree2'] = 'une semaine'
df_prod42_test2.loc[(df_prod42_test2['duree_souscription'] > 9) & (df_prod42_test2['duree_souscription'] < 17), 'duree2'] = 'deux semaines'
df_prod42_test2.loc[((df_prod42_test2['duree_souscription'] > 25) & (df_prod42_test2['duree_souscription'] < 33)), 'duree2'] = 'un mois'
#df_prod42_test2.loc[((df_prod42_test2['duree_souscription'] > 50) & (df_prod42_test2['duree_souscription'] < 70)), 'duree2'] = 'deux mois'
df_prod42_test2.loc[((df_prod42_test2['duree_souscription'] > 80) & (df_prod42_test2['duree_souscription'] < 100)), 'duree2'] = 'trois mois'
#df_prod42_test2.loc[((df_prod42_test2['duree_souscription'] > 110) & (df_prod42_test2['duree_souscription'] < 130)), 'duree2'] = 'quatres mois'
df_prod42_test2.loc[((df_prod42_test2['duree_souscription'] > 140) & (df_prod42_test2['duree_souscription'] < 160)), 'duree2'] = 'cinq mois'
df_prod42_test2.loc[((df_prod42_test2['duree_souscription'] > 170) & (df_prod42_test2['duree_souscription'] < 200)), 'duree2'] = 'six mois'
df_prod42_test2.loc[df_prod42_test2['duree_souscription'] > 200, 'duree2'] = 'plus de 6 mois'

plt.rcParams['font.size'] = 10.0
taille = len(df_prod42_test2.duree2.value_counts())
cs=plt.cm.Set1(0.7*np.arange(taille)/taille)

values = (df_prod42_test2.duree2.value_counts())

#plt.figure(figsize=(6,6))

ax = plt.subplot(121)

df_prod42_test2.duree2.value_counts(sort = False).plot(kind='pie' 
       ,colors = cs, autopct=make_autopct(values),labeldistance=1.02, pctdistance=0.68,radius=1.02)

ax.axes.get_yaxis().set_visible(False)
ax.set_title('duree de la souscription des nouveaux abonnes 2015')

plt.axis('equal') 
plt.tight_layout()

df_prod42_test2 = df_prod42_test2.loc[((df_prod42_test2.service_Abo_Equipe_payant ==1) 
                                       |(df_prod42_test2.service_Abo_via_tiers_payant ==1))]
df_prod42_test2.duree2 = np.nan
df_prod42_test2.loc[df_prod42_test2['duree_souscription'] < 9, 'duree2'] = 'une semaine'
df_prod42_test2.loc[(df_prod42_test2['duree_souscription'] > 9) & (df_prod42_test2['duree_souscription'] < 17), 'duree2'] = 'deux semaines'
df_prod42_test2.loc[((df_prod42_test2['duree_souscription'] > 25) & (df_prod42_test2['duree_souscription'] < 33)), 'duree2'] = 'un mois'
df_prod42_test2.loc[((df_prod42_test2['duree_souscription'] > 50) & (df_prod42_test2['duree_souscription'] < 70)), 'duree2'] = 'deux mois'
#df_prod42_test2.loc[((df_prod42_test2['duree_souscription'] > 80) & (df_prod42_test2['duree_souscription'] < 100)), 'duree2'] = 'trois mois'
df_prod42_test2.loc[((df_prod42_test2['duree_souscription'] > 110) & (df_prod42_test2['duree_souscription'] < 130)), 'duree2'] = 'quatres mois'
df_prod42_test2.loc[((df_prod42_test2['duree_souscription'] > 140) & (df_prod42_test2['duree_souscription'] < 160)), 'duree2'] = 'cinq mois'
df_prod42_test2.loc[((df_prod42_test2['duree_souscription'] > 170) & (df_prod42_test2['duree_souscription'] < 200)), 'duree2'] = 'six mois'
df_prod42_test2.loc[df_prod42_test2['duree_souscription'] > 200, 'duree2'] = 'plus de 6 mois'


plt.rcParams['font.size'] = 10.0
taille = len(df_prod42_test2.duree2.value_counts())
cs=plt.cm.Set1(0.7*np.arange(taille)/taille)

values = (df_prod42_test2.duree2.value_counts())

ax = plt.subplot(122)

df_prod42_test2.duree2.value_counts(sort = False).plot(kind='pie' 
       ,colors = cs, autopct=make_autopct(values),labeldistance=1.02, pctdistance=0.65,radius=1.02)

ax.axes.get_yaxis().set_visible(False)
ax.set_title('duree de la souscription des nouveaux abonnes payant 2015')


plt.axis('equal') 
plt.tight_layout()
plt.show()

In [16]:
df_prod42_test2 = pd.read_pickle(path+"\s_final_groupby_dataframe_sub_2015.pkl")
#-------------------------- pie pour la durée l'abonnement OK--------------------------

df_prod42_test2 = df_prod42_test2.loc[df_prod42_test2.somme_mensualites >1]

df_prod42_test2["duree2"] = df_prod42_test2.duree_abonnement_reelle
df_prod42_test2["duree_souscription"] = df_prod42_test2.duree_abonnement_reelle

df_prod42_test2.duree2 = np.nan
df_prod42_test2.loc[df_prod42_test2['duree_souscription'] < 9, 'duree2'] = 'une semaine'
df_prod42_test2.loc[(df_prod42_test2['duree_souscription'] > 9) & (df_prod42_test2['duree_souscription'] < 17), 'duree2'] = 'deux semaines'
df_prod42_test2.loc[((df_prod42_test2['duree_souscription'] > 25) & (df_prod42_test2['duree_souscription'] < 33)), 'duree2'] = 'un mois'
df_prod42_test2.loc[((df_prod42_test2['duree_souscription'] > 50) & (df_prod42_test2['duree_souscription'] < 70)), 'duree2'] = 'deux mois'
df_prod42_test2.loc[((df_prod42_test2['duree_souscription'] > 80) & (df_prod42_test2['duree_souscription'] < 100)), 'duree2'] = 'trois mois'
df_prod42_test2.loc[((df_prod42_test2['duree_souscription'] > 110) & (df_prod42_test2['duree_souscription'] < 130)), 'duree2'] = 'quatres mois'
df_prod42_test2.loc[((df_prod42_test2['duree_souscription'] > 140) & (df_prod42_test2['duree_souscription'] < 160)), 'duree2'] = 'cinq mois'
df_prod42_test2.loc[((df_prod42_test2['duree_souscription'] > 170) & (df_prod42_test2['duree_souscription'] < 200)), 'duree2'] = 'six mois'
df_prod42_test2.loc[((df_prod42_test2['duree_souscription'] > 300) & (df_prod42_test2['duree_souscription'] < 380)), 'duree2'] = 'un an'

plt.rcParams['font.size'] = 10.0
taille = len(df_prod42_test2.duree2.value_counts())
cs=plt.cm.Set1(0.7*np.arange(taille)/taille)

values = (df_prod42_test2.duree2.value_counts())

#plt.figure(figsize=(6,6))

ax = plt.subplot(111)

df_prod42_test2.duree2.value_counts(sort = False).plot(kind='pie' 
       ,colors = cs, autopct=make_autopct(values),labeldistance=1.02, pctdistance=0.68,radius=1.02)

ax.axes.get_yaxis().set_visible(False)
ax.set_title('duree de la souscription des nouveaux abonnes 2015 ayant paye plus de 1 euro')


plt.axis('equal') 
plt.tight_layout()
plt.show()

In [20]:
#-------------------------- pie pour le prix mensuel de l'abonnement en fonction de sa durée --------------------------
plt.figure(figsize=(8,8)) 
df_prod42_test2["ExplicitPrice"] = df_prod42_test2.prix_mensuel
#-- un mois--
taille = len(df_prod42_test2.loc[df_prod42_test2["duree2"]=="un mois"].ExplicitPrice.value_counts(sort=False))
cs=plt.cm.Set1(0.7*np.arange(taille)/taille)

values = (df_prod42_test2.loc[df_prod42_test2["duree2"]=="un mois"].ExplicitPrice.value_counts())

ax= plt.subplot(311)

df_prod42_test2.loc[df_prod42_test2["duree2"]=="un mois"].ExplicitPrice.value_counts().plot(kind='pie' 
       ,colors = cs, autopct=make_autopct(values),labeldistance=1.05, pctdistance=0.65,radius=1.02)
    
plt.title('un mois')
ax.axes.get_yaxis().set_visible(False)
plt.axis('equal')
plt.tight_layout()


##--deux mois--
#taille = len(df_prod42_test2.loc[df_prod42_test2["duree2"]=="deux mois"].ExplicitPrice.value_counts(sort=False))
#cs=plt.cm.Set1(0.7*np.arange(taille)/taille)
#
#values = (df_prod42_test2.loc[df_prod42_test2["duree2"]=="deux mois"].ExplicitPrice.value_counts())
#
#ax= plt.subplot(322)
#
#df_prod42_test2.loc[df_prod42_test2["duree2"]=="deux mois"].ExplicitPrice.value_counts().plot(kind='pie' 
#       ,colors = cs, autopct=make_autopct(values),labeldistance=1.05, pctdistance=0.75,radius=1.02)
#    
#plt.title('deux mois')
#ax.axes.get_yaxis().set_visible(False)
#plt.axis('equal')
#plt.tight_layout()
#
#--trois mois--
taille = len(df_prod42_test2.loc[df_prod42_test2["duree2"]=="trois mois"].ExplicitPrice.value_counts(sort=False))
cs=plt.cm.Set1(0.7*np.arange(taille)/taille)

values = (df_prod42_test2.loc[df_prod42_test2["duree2"]=="trois mois"].ExplicitPrice.value_counts())

ax= plt.subplot(312)

df_prod42_test2.loc[df_prod42_test2["duree2"]=="trois mois"].ExplicitPrice.value_counts().plot(kind='pie' 
       ,colors = cs, autopct=make_autopct(values),labeldistance=1.05, pctdistance=0.65,radius=1.02)
    
plt.title('trois mois')
ax.axes.get_yaxis().set_visible(False)
plt.axis('equal')
plt.tight_layout()


#--quatres mois--
#taille = len(df_prod42_test2.loc[df_prod42_test2["duree2"]=="quatres mois"].ExplicitPrice.value_counts(sort=False))
#cs=plt.cm.Set1(0.7*np.arange(taille)/taille)
#
#values = (df_prod42_test2.loc[df_prod42_test2["duree2"]=="quatre mois"].ExplicitPrice.value_counts())
#
#ax= plt.subplot(324)
#
#df_prod42_test2.loc[df_prod42_test2["duree2"]=="quatres mois"].ExplicitPrice.value_counts().plot(kind='pie' 
#       ,colors = cs, autopct=make_autopct(values),labeldistance=1.05, pctdistance=0.75,radius=1.02)
#    
#plt.title('quatres mois')
#ax.axes.get_yaxis().set_visible(False)
#plt.axis('equal')
#plt.tight_layout()


##--cinq mois--
#taille = len(df_prod42_test2.loc[df_prod42_test2["duree2"]=="cinq mois"].ExplicitPrice.value_counts(sort=False))
#cs=plt.cm.Set1(0.7*np.arange(taille)/taille)
#
#values = (df_prod42_test2.loc[df_prod42_test2["duree2"]=="cinq mois"].ExplicitPrice.value_counts())
#
#ax= plt.subplot(325)
#
#df_prod42_test2.loc[df_prod42_test2["duree2"]=="cinq mois"].ExplicitPrice.value_counts().plot(kind='pie' 
#       ,colors = cs, autopct=make_autopct(values),labeldistance=1.05, pctdistance=0.75,radius=1.02)
#    
#plt.title('cinq mois')
#ax.axes.get_yaxis().set_visible(False)
#plt.axis('equal')
#plt.tight_layout()


#--six mois--
taille = len(df_prod42_test2.loc[df_prod42_test2["duree2"]=="six mois"].ExplicitPrice.value_counts(sort=False))
cs=plt.cm.Set1(0.7*np.arange(taille)/taille)

values = (df_prod42_test2.loc[df_prod42_test2["duree2"]=="six mois"].ExplicitPrice.value_counts())

ax= plt.subplot(313)

df_prod42_test2.loc[df_prod42_test2["duree2"]=="six mois"].ExplicitPrice.value_counts().plot(kind='pie' 
       ,colors = cs, autopct=make_autopct(values),labeldistance=1.05, pctdistance=0.65,radius=1.02)
    
plt.title('six mois')
ax.axes.get_yaxis().set_visible(False)
plt.axis('equal')
plt.tight_layout()
plt.show()


In [18]:
df_prod42_test2 = pd.read_pickle(path+"\s_final_groupby_dataframe_sub_2015.pkl")
#-------------------------- pie les répartions des sommes payées par les abonnement --------------------------

df_prod42_test2["somme2"] = df_prod42_test2.somme_mensualites

df_prod42_test2.somme2 = np.nan
df_prod42_test2.loc[df_prod42_test2['somme_mensualites'] == 0, 'somme2'] = '0€'
df_prod42_test2.loc[(df_prod42_test2['somme_mensualites'] > 0) & (df_prod42_test2['somme_mensualites'] < 1), 'somme2'] = 'moins de 1€'
#df_prod42_test2.loc[(df_prod42_test2['somme_mensualites'] >=1) & (df_prod42_test2['somme_mensualites'] < 2), 'somme2'] = 'moins de 2€'
#df_prod42_test2.loc[(df_prod42_test2['somme_mensualites'] >= 2) & (df_prod42_test2['somme_mensualites'] < 5), 'somme2'] = 'entre 1€ et 5€'
df_prod42_test2.loc[(df_prod42_test2['somme_mensualites'] >= 1) & (df_prod42_test2['somme_mensualites'] < 10), 'somme2'] = 'entre 1€ et 10€'
df_prod42_test2.loc[(df_prod42_test2['somme_mensualites'] >= 10) & (df_prod42_test2['somme_mensualites'] < 20), 'somme2'] = 'entre 10€ et 20€'
df_prod42_test2.loc[(df_prod42_test2['somme_mensualites'] >= 20) & (df_prod42_test2['somme_mensualites'] < 100), 'somme2'] = 'entre 20€ et 100€'
df_prod42_test2.loc[(df_prod42_test2['somme_mensualites'] >= 100), 'somme2'] = 'plus de 100€'


plt.rcParams['font.size'] = 10.0
taille = len(df_prod42_test2.somme2.value_counts())
cs=plt.cm.Set1(0.7*np.arange(taille)/taille)

values = (df_prod42_test2.somme2.value_counts())

#plt.figure(figsize=(6,6))

ax = plt.subplot(111)

df_prod42_test2.somme2.value_counts(sort = False).plot(kind='pie' 
       ,colors = cs, autopct=make_autopct(values),labeldistance=1.02, pctdistance=0.68,radius=1.02)

ax.axes.get_yaxis().set_visible(False)
ax.set_title('montant paye par les nouveaux abonnes 2015')


plt.axis('equal') 
plt.tight_layout()
plt.show()